<a href="https://colab.research.google.com/github/sunny9sinha/TSAI_Session_3/blob/main/Session_3_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch as torch
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from random import randint
import torch.optim as optim

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
train_dataset = datasets.MNIST(
    root ='.',
    train=True,
    transform = transforms.ToTensor(),
    download=True,
)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
train_dataset.data.shape

torch.Size([60000, 28, 28])

In [5]:
test_dataset = datasets.MNIST(
    root='.',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

In [6]:
test_dataset.data.shape

torch.Size([10000, 28, 28])

In [7]:
train_dataset[0]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [8]:
type(np.array(train_dataset.data))

numpy.ndarray

In [9]:
len(train_dataset.data)

60000

In [10]:
train_dataset.data.shape

torch.Size([60000, 28, 28])

In [11]:
train_dataset.data[0].shape

torch.Size([28, 28])

In [12]:
from torch.utils.data import Dataset
import pandas as pd
class customMNISTDataset(Dataset):

  def __init__(self,dat):
      self.X = dat.data
      self.Y = dat.targets
  
  def __len__(self):
    return len(self.X)
  
  def __getitem__(self,idx):
    #img_data = self.X
    #img_data = np.array(img_data).astype(np.uint8).reshape(28,28,1)
    img_data = self.X[idx]
    #img_data = np.array([i for i in np.array(self.X)]).reshape(1,28,28) 
    #rand_no = np.array([randint(0,9) for i in range(0,len(self.Y))])
    rand_no = randint(0,9)
    y1 = self.Y[idx]
    y2 = y1+rand_no
    return img_data, rand_no, y1,y2


In [13]:
#transform = transforms.Compose([transforms.ToTensor()])
dataset_train = customMNISTDataset(train_dataset)
print('Number of samples in train dataset: ', len(dataset_train))
#print(dataset_train[0:5])

Number of samples in train dataset:  60000


In [14]:
for i in dataset_train:
  image,randnum,label,numsum = i
  print(image.shape, randnum, label, numsum)
  break;

torch.Size([28, 28]) 5 tensor(5) tensor(10)


In [62]:
import torch.nn.functional as F
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120,bias=True)
        self.fc2 = nn.Linear(in_features=120, out_features=60,bias=True)
        self.fc3 = nn.Linear(in_features=60, out_features=10,bias=True) ## MNIST output

        ## MNIST output and Random int
        self.fc4 = nn.Linear(in_features = 10, out_features = 30,bias=True)
        self.fc5 = nn.Linear(in_features = 30, out_features = 35,bias=True)
        self.out = nn.Linear(in_features = 35, out_features = 19,bias=True)



    def forward(self, t, rand_num):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # (6) hidden linear layer
        t_out = self.fc3(t)
        t1 = t_out
        t1 = F.softmax(t1,dim=1)

        ## (7) hidden linear layer
        #rand_num = torch.cat((t_out,rand_num), dim =1)
        #rand_num = torch.Tensor(rand_num)
        #rand_num = torch.cat((t_out,rand_num.reshape(-1,1)), dim =1)
        t_out = t1.argmax(dim=1)
        t_out = F.one_hot(torch.tensor(t_out), num_classes=10).float()
        rand_num = t_out.add(rand_num)

        x = self.fc4(rand_num)
        x = F.relu(x)

        ## (8) hidden linear layer
        x = self.fc5(x)
        x = F.relu(x)

        ## (9) output layer for sum
        x = self.out(x)
        
        return x, t1,F.softmax(x, dim=1)

In [63]:
network = Network()
network = network.to(device)
optimizer = optim.Adam(network.parameters(), lr=0.01)
train_loader = torch.utils.data.DataLoader(
     dataset_train
    ,batch_size=10
)

In [17]:
batch = next(iter(train_loader))

len(batch), type(batch)

(4, list)

In [18]:
images,randnum,labels,numsum = batch

In [19]:
images.shape, labels.shape, randnum.shape, numsum.shape

(torch.Size([10, 28, 28]),
 torch.Size([10]),
 torch.Size([10]),
 torch.Size([10]))

In [20]:
images.unsqueeze(1).shape

torch.Size([10, 1, 28, 28])

In [21]:
images.unsqueeze(1).float()[0].shape

torch.Size([1, 28, 28])

In [69]:
optimizer = optim.Adam(network.parameters(), lr=0.1)
for epoch in range(10):

    total_loss_label = 0
    total_loss_sum = 0


    for batch in train_loader: # Get Batch
        images,rand_num,labels,num_sum = batch 
        images = images.to(device)
        rand_num = rand_num.to(device)
        labels = labels.to(device)
        num_sum = num_sum.to(device)
        # print(F.one_hot(torch.tensor(rand_num), num_classes=10).float())

        pred, pred_label, pred_sum = network(images.unsqueeze(1).float(), F.one_hot(torch.tensor(rand_num), num_classes=10).float()) # Pass Batch
        #pred_sum = pred_sum.cpu().detach().numpy()
        #print(pred_sum.shape)
        #pred_sum = [round( pred_sum[i,0]* float(9 * 2)) for i in range(10) ]
        loss_label = F.cross_entropy(pred_label, labels) # Calculate Loss
        #print(pred_sum.argmax(dim=1).shape)
        #print(num_sum.shape)
        #print(type(pred_sum))
        #print(type(num_sum))
        #loss_sum = 0
        loss_sum = F.cross_entropy(pred_sum,num_sum)
        #F.mse_loss(pred_sum.argmax(dim=1).float(),num_sum.float())

        optimizer.zero_grad()
        loss = loss_label + loss_sum
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss_label += loss_label.item()
        total_loss_sum += loss_sum

        # for i in range(10):
        #   print('Pred Label: ',pred_label[i,])
        #   print('Actual Label: ', labels[i,])
        #   print('Pred Sum: ', pred_sum.argmax(dim=1)[i])
        #   print('Actual Sum: ', num_sum[i])


    print(
        "epoch", epoch, 
        "loss_label: ", total_loss_label,
        "loss_sum: ", total_loss_sum
    )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


epoch 0 loss_label:  14174.60205590725 loss_sum:  tensor(17489.1289, device='cuda:0', grad_fn=<AddBackward0>)
epoch 1 loss_label:  14174.60205590725 loss_sum:  tensor(17478.2109, device='cuda:0', grad_fn=<AddBackward0>)
epoch 2 loss_label:  14174.60205590725 loss_sum:  tensor(17476.8848, device='cuda:0', grad_fn=<AddBackward0>)
epoch 3 loss_label:  14174.60205590725 loss_sum:  tensor(17476.7266, device='cuda:0', grad_fn=<AddBackward0>)
epoch 4 loss_label:  14174.60205590725 loss_sum:  tensor(17475.9199, device='cuda:0', grad_fn=<AddBackward0>)
epoch 5 loss_label:  14174.60205590725 loss_sum:  tensor(17474.2422, device='cuda:0', grad_fn=<AddBackward0>)
epoch 6 loss_label:  14174.60205590725 loss_sum:  tensor(17476.1035, device='cuda:0', grad_fn=<AddBackward0>)
epoch 7 loss_label:  14174.60205590725 loss_sum:  tensor(17483.1523, device='cuda:0', grad_fn=<AddBackward0>)
epoch 8 loss_label:  14174.60205590725 loss_sum:  tensor(17473.7324, device='cuda:0', grad_fn=<AddBackward0>)
epoch 9 lo